# Data Mining & Analytics: Kaggle Competition


---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# If you are using Google Colab, upload the proper files from Kaggle. 
# If you are using Google Colab, you can use ls to help you figure out what directory you are in.
train = pd.read_csv('sample_data/train.csv')
test = pd.read_csv('sample_data/test.csv')
#baseline = pd.read_csv('sample_data/gender_submission.csv')

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
#baseline.head()

Data Exploration

In [ ]:
train[['Pclass','Survived']].groupby('Pclass').sum('Survived').reset_index()

,Pclass,Survived
0,1,136
1,2,87
2,3,119


In [ ]:
train[['Cabin', 'Survived']].groupby('Cabin').sum().reset_index()

,Cabin,Survived
0,A10,0
1,A14,0
2,A16,1
3,A19,0
4,A20,1
...,...,...
142,F33,3
143,F38,0
144,F4,2
145,G6,2


In [ ]:
train[['SibSp', 'Survived']].groupby('SibSp').sum().reset_index()

,SibSp,Survived
0,0,210
1,1,112
2,2,13
3,3,4
4,4,3
5,5,0
6,8,0


In [ ]:
train[['Parch', 'Survived']].groupby('Parch').sum().reset_index()

,Parch,Survived
0,0,233
1,1,65
2,2,40
3,3,3
4,4,0
5,5,1
6,6,0


In [ ]:
train_test = [train, test]

In [ ]:
# binning age to make it easier to work with
for dataset in train_test:
  dataset['Age'].fillna(dataset['Age'].dropna().mean(), axis=0, inplace=True)
  dataset.loc[ dataset['Age'] <= 4, 'Age'] = 0
  dataset.loc[(dataset['Age'] > 4) & (dataset['Age'] <= 15), 'Age'] = 1
  dataset.loc[(dataset['Age'] > 15) & (dataset['Age'] <= 25), 'Age'] = 2
  dataset.loc[(dataset['Age'] > 25) & (dataset['Age'] <= 35), 'Age'] = 3
  dataset.loc[(dataset['Age'] > 35) & (dataset['Age'] <= 65), 'Age'] = 4
  dataset.loc[(dataset['Age'] > 65) & (dataset['Age'] <= 80), 'Age'] = 5
  dataset.loc[ dataset['Age'] > 80, 'Age'] = 6
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,2.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,4.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,3.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,3.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,3.0,0,0,373450,8.0500,NaN,S


In [ ]:
fare_first = np.percentile(train['Fare'], 25)
fare_second = np.percentile(train['Fare'], 50)
fare_third = np.percentile(train['Fare'], 75)
fare_first, fare_second, fare_third

(7.9104, 14.4542, 31.0)

In [ ]:
# binning fare to make it easier to work with
for dataset in train_test:
  dataset.fillna(dataset['Fare'].dropna().mean())
  dataset.loc[ dataset['Fare'] <= 7.9104, 'Fare'] = 0
  dataset.loc[(dataset['Fare'] > 7.9104) & (dataset['Fare'] <= 14.4542), 'Fare'] = 1
  dataset.loc[(dataset['Fare'] > 14.4542) & (dataset['Fare'] <= 31.0), 'Fare'] = 2
  dataset.loc[ dataset['Fare'] > 31.0, 'Fare'] = 3
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,2.0,1,0,A/5 21171,0.0,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,4.0,1,0,PC 17599,3.0,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,3.0,0,0,STON/O2. 3101282,1.0,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,3.0,1,0,113803,3.0,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,3.0,0,0,373450,1.0,NaN,S


In [ ]:
train[['Fare','Survived']].groupby('Fare').sum().reset_index()

,Fare,Survived
0,0.0,44
1,1.0,68
2,2.0,101
3,3.0,129


Preprocessing

In [ ]:
from sklearn import preprocessing, tree
le = preprocessing.LabelEncoder()
le.fit(['male', 'female'])
train['Sex'] = le.fit_transform(train['Sex'])
test['Sex'] = le.fit_transform(test['Sex'])

le.fit(["Q", "S", "C"])
train["Embarked"] = le.fit_transform(train["Sex"])
test["Embarked"] = le.fit_transform(test["Sex"])


train["SibSp"] = train["SibSp"].fillna(np.mean(train["SibSp"]))
train["Parch"] = train["Parch"].fillna(np.mean(train["Parch"]))
train["Family"] = train["Parch"] + train["SibSp"]
# train.loc[ train['Family'] == 0, 'Family'] = 0
# train.loc[train['Family'] == 1, 'Family'] = 1
# train.loc[train['Family'] == 2, 'Family'] = 2
train["Single"] = train["Family"].apply(lambda x: 1 if x == 0 else 0)
train["Small_Family"] = train["Family"].apply(lambda x: 1 if x == 1 else 0)
train["Big_Family"] = train["Family"].apply(lambda x: 1 if x >= 2 else 0)

test["SibSp"] = test["SibSp"].fillna(np.mean(test["SibSp"]))
test["Parch"] = test["Parch"].fillna(np.mean(test["Parch"]))
test["Family"] = test["Parch"] + test["SibSp"]
# test.loc[test['Family'] == 0, 'Family'] = 0
# test.loc[test['Family'] == 1, 'Family'] = 1
# test.loc[test['Family'] == 2, 'Family'] = 2
test["Single"] = test["Family"].apply(lambda x: 1 if x == 0 else 0)
test["Small_Family"] = test["Family"].apply(lambda x: 1 if x == 1 else 0)
test["Big_Family"] = test["Family"].apply(lambda x: 1 if x >= 2 else 0)

for name_string in train['Name']:
  train['Title'] = train['Name'].str.extract('([A-Za-z]+)\.', expand=True)
print(train["Title"].unique())

train = train.replace("Major", "Col").replace("Don", "Mr").replace("Mme", "Mrs").replace("Lady", "Ms").replace("Sir", "Mr").replace("Mlle", "Miss").replace("Capt", "Col").replace("Countess", "Ms").replace("Jonkheer", "Ms")

print(train["Title"].unique())

titles = {"Col":0, "Mr":1, "Mrs":2, "Ms":3, "Miss":4, "Master":5}
train['Title'] = train['Title'].map(titles)
train['Title'] = train['Title'].fillna(0)

# train_ohe_titles = pd.get_dummies(train["Title"])

for name_string in test["Name"]:
  test["Title"] = test["Name"].str.extract("([A-Za-z]+)\.", expand = True)

test = test.replace("Dona", "Ms")
print(test["Title"].unique())

test['Title'] = test['Title'].map(titles)
test['Title'] = test['Title'].fillna(0)
# test_ohe_titles = pd.get_dummies(test["Title"])

# test = test.join(test_ohe_titles)
# train = train.join(train_ohe_titles)
# #you can't run the above cells multiople times in one runtime
# test.head()
train

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'Countess' 'Jonkheer']
['Mr' 'Mrs' 'Miss' 'Master' 'Rev' 'Dr' 'Ms' 'Col']
['Mr' 'Mrs' 'Miss' 'Master' 'Ms' 'Col' 'Rev' 'Dr']


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family,Single,Small_Family,Big_Family,Title
0,1,0,3,"Braund, Mr. Owen Harris",1,2.0,1,0,A/5 21171,0.0,NaN,1,1,0,1,0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,4.0,1,0,PC 17599,3.0,C85,0,1,0,1,0,2.0
2,3,1,3,"Heikkinen, Miss. Laina",0,3.0,0,0,STON/O2. 3101282,1.0,NaN,0,0,1,0,0,4.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,3.0,1,0,113803,3.0,C123,0,1,0,1,0,2.0
4,5,0,3,"Allen, Mr. William Henry",1,3.0,0,0,373450,1.0,NaN,1,0,1,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,3.0,0,0,211536,1.0,NaN,1,0,1,0,0,0.0
887,888,1,1,"Graham, Miss. Margaret Edith",0,2.0,0,0,112053,2.0,B42,0,0,1,0,0,4.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,3.0,1,2,W./C. 6607,2.0,NaN,0,3,0,0,1,4.0
889,890,1,1,"Behr, Mr. Karl Howell",1,3.0,0,0,111369,2.0,C148,1,0,1,0,0,1.0


In [ ]:
train_df = train.drop(columns = ["PassengerId", "Survived", "Name", "Ticket", "Cabin", "Family"])  
for n in range (0, len(train_df.columns) - 1):
  mean = np.mean(train_df.iloc[:n])
  train_df.iloc[:n] = train_df.iloc[:n].fillna(mean)
#train_df["Age"] = train_df["Age"].fillna(np.mean(train_df["Age"]))
#train_df["Fare"] = np.log(train_df["Fare"], where = train_df["Fare"] > 0)
train_dff = train_df
train_dff

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Single,Small_Family,Big_Family,Title
0,3,1,2.0,1,0,0.0,1,0,1,0,1.0
1,1,0,4.0,1,0,3.0,0,0,1,0,2.0
2,3,0,3.0,0,0,1.0,0,1,0,0,4.0
3,1,0,3.0,1,0,3.0,0,0,1,0,2.0
4,3,1,3.0,0,0,1.0,1,1,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
886,2,1,3.0,0,0,1.0,1,1,0,0,0.0
887,1,0,2.0,0,0,2.0,0,1,0,0,4.0
888,3,0,3.0,1,2,2.0,0,0,0,1,4.0
889,1,1,3.0,0,0,2.0,1,1,0,0,1.0


In [ ]:
test_df = test.drop(columns = ["PassengerId", "Name", "Ticket", "Cabin", "Family"]) 
for n in range (0, len(test_df.columns) - 1):
  mean = np.mean(test_df.iloc[:n])
  test_df.iloc[:n] = test_df.iloc[:n].fillna(mean)
test_df["Fare"] = test_df["Fare"].fillna(np.mean(test_df["Fare"]))
#test_df["Fare"] = np.log(test_df["Fare"], where = test_df["Fare"] > 0)
#test_df["Age"] = test_df["Age"].fillna(np.mean(test_df["Age"]))
print(test_df.isna().sum())
test_dff = test_df
test_dff

Pclass          0
Sex             0
Age             0
SibSp           0
Parch           0
Fare            0
Embarked        0
Single          0
Small_Family    0
Big_Family      0
Title           0
dtype: int64


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Single,Small_Family,Big_Family,Title
0,3,1,3.0,0,0,0.0,1,1,0,0,1.0
1,3,0,4.0,1,0,0.0,0,0,1,0,2.0
2,2,1,4.0,0,0,1.0,1,1,0,0,1.0
3,3,1,3.0,0,0,1.0,1,1,0,0,1.0
4,3,0,2.0,1,1,1.0,0,0,0,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...
413,3,1,3.0,0,0,1.0,1,1,0,0,1.0
414,1,0,4.0,0,0,3.0,0,1,0,0,3.0
415,3,1,4.0,0,0,0.0,1,1,0,0,1.0
416,3,1,3.0,0,0,1.0,1,1,0,0,1.0


Logistic Regression

In [ ]:
X_train = train_dff
Y_train = train["Survived"]
X_test  = test_df

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
Y_pred

array([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

Decision Tree

In [ ]:
from sklearn.feature_extraction import DictVectorizer

df_dict = train_dff.to_dict("records")
test_dict = test_dff.to_dict("records")

vec = DictVectorizer()
df_dict = vec.fit_transform(df_dict).toarray()
test_dict = vec.fit_transform(test_dict).toarray()

target = train["Survived"].tolist()

print(df_dict)
print(target)
X = df_dict
y = target
clf = tree.DecisionTreeClassifier(max_depth=3)
clf = clf.fit(X, y)
y_pred = clf.predict(test_dict)

y_pred

[[2. 0. 1. ... 0. 1. 1.]
 [4. 0. 0. ... 0. 1. 2.]
 [3. 0. 0. ... 1. 0. 4.]
 ...
 [3. 1. 0. ... 0. 0. 4.]
 [3. 0. 1. ... 1. 0. 1.]
 [3. 0. 1. ... 1. 0. 1.]]
[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

K Nearest Neighbors

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X, y)
Y_pred = knn.predict(test_dict)
Y_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

Random Forest Classifier

In [ ]:
X = df_dict
y = target
clf = RandomForestClassifier(n_estimators=65, max_depth=3)
clf = clf.fit(X, y)
y_pred = clf.predict(test_dict)

#print(clf.score(X, y))
y_pred

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

Linear Regression

In [ ]:
# from sklearn.linear_model import LinearRegression

# train_sexageticket = train[["Sex", "Pclass", "SibSp", "Parch"]]
# test_sexageticket = test[["Sex", "Pclass", "SibSp", "Parch"]]

# x_train = train_sexageticket
# y_train = train['Survived']
# x_test = test_sexageticket

# model = LinearRegression()
# model.fit(x_train, y_train)
# y_pred = (np.round(model.predict(x_test)))
 
# y_pred

What is the majority class of `Survived`?

In [ ]:
train['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

Let's predict this as a naive classifier. 

In [ ]:
# actual = train['Survived']
# prediction = y_pred
# # accuracy_sc/ore(y_true=actual, y_pred=prediction)
# accuracy_score(actual, prediction)

Let's do this for our `test` dataframe as well. 

In [ ]:
test['Survived'] = 0

In [ ]:
Data = {"Survived": y_pred}
final_prediction = pd.DataFrame(Data, test["PassengerId"])
#final_prediction["Survived"] = y_pred
final_prediction['Survived'] = final_prediction['Survived'].astype(int)
final_prediction.head()


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


To create a submission for the Kaggle Competition, use `to_csv` with the name of your file output. An example is below. Your file will be output in your current directory. On Google Colab, by default, this is `content`.

In [ ]:
final_prediction.to_csv('my_prediction.csv')

Download this csv and upload it to Kaggle. Good luck! We are rooting for you :)